In [ ]:
yolo detect train \
    data=data.yaml \
    model=yolov8n.pt \
    epochs=100 \
    imgsz=640 \
    batch=8 \
    workers=2 \
    device=0 \
    cache=ram \
    exist_ok=True

## box_loss: 0.945 

In [ ]:
nohup yolo detect train \
    data=data.yaml \
    model=yolov8s.pt \
    epochs=100 \
    imgsz=0 \
    batch=8 \              
    workers=2 \            
    device=0 \
    lr0=0.01 \
    patience=10 \
    cache=False \          
    augment=True \
    verbose=True > train_repaired.log 2>&1 &

## box_loss: 0.8774


In [ ]:
nohup yolo detect train \
    resume=True \
    model=runs/detect/train4/weights/last.pt \
    data=data.yaml \
    epochs=100 \
    imgsz=640 \
    batch=4 \             
    workers=1 \            
    device=0 \
    lr0=0.01 \
    patience=10 \
    cache=False \          
    augment=False \        
    val=False \           
    verbose=True > train_light.log 2>&1 &

In [ ]:
yolo train \
  model=yolov11l.pt \
  data=data.yaml \  
  epochs=100 \
  imgsz=480 \
  batch=16 \
  workers=2 \            
  device=0 \
  lr0=0.005 \
  patience=10 \
  cache=False \          
  augment=False \        
  val=False \ 
  project=yolov11_subset_results

In [ ]:
yolo val model=yolov11_subset_results/lr001/weights/best.pt data=data.yaml conf=0.1 iou=0.5 plots=True workers=0 cache=False

In [ ]:
yolo train model=yolo11l data=data.yaml epochs=100 imgsz=640 batch=8 workers=0 device=0 lr0=0.005 augment=True optimizer="AdamW" cls=0.5 pretrained=True  